In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

max_seq_length = 4096  # 최대 시퀀스 길이를 설정합니다.
dtype = torch.float16 
load_in_4bit = True  # 4bit 양자화 로드 여부를 설정합니다.

model_name = "Suchae/test-finetuned-merge"  # 모델 이름을 설정합니다.

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=dtype
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    torch_dtype=dtype
)

tokenizer = AutoTokenizer.from_pretrained(model_name)


c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\quantizers\auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

In [3]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = EOS_TOKEN  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.


# # 패딩 토큰을 eos_token으로 설정합니다.
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

In [4]:
# 바뀐 모델
from transformers import TextStreamer

# 입력 텍스트를 설정합니다.
input_text = alpaca_prompt.format(
    "한국은 영국 정부와 몇 개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니까?",  # 지시사항
    "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
)
inputs = tokenizer([input_text], return_tensors="pt", padding=True).to('cuda')

# 생성된 텍스트를 스트리밍할 TextStreamer를 설정합니다.
text_streamer = TextStreamer(tokenizer)

# 텍스트 생성을 시작합니다.
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다. 필요시 직접 정의할 수 있습니다.
)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
한국은 영국 정부와 몇 개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니까?

### Response:


c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyboardInterrupt: 

In [13]:
# 모델 삭제
del model

# GPU 메모리 캐시 비우기
torch.cuda.empty_cache()

import gc

# 가비지 컬렉션 실행
gc.collect()

# GPU 메모리 캐시 비우기
torch.cuda.empty_cache()

NameError: name 'model' is not defined